In [ ]:
from snp_query_box.dsnp_transform import pbgGroupTransform

In [ ]:
import os
USERNAME = os.getenv('USERNAME').lower()
new_pull = False

In [ ]:
import os
import pandas as pd
import datetime as dt
import time
import warnings
from dateutil.relativedelta import relativedelta


from snp_query_box import DsnpHelperFunction, snpQueries

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [ ]:
print(f"User: {USERNAME} Started creating pull_vbc and pbg_group_clean_df in the shared storage")

In [ ]:
start_time = time.time()

In [ ]:
today = dt.datetime.today()
today_str = today.strftime('%Y-%m-%d')
last_date_of_last_month_str = DsnpHelperFunction.last_date_of_last_month(today_str, output_type="string")
last_date_of_last_month = DsnpHelperFunction.last_date_of_last_month(today_str)

last_month = last_date_of_last_month_str[:7]
one_year_rolling_back_date = pd.Timestamp(last_date_of_last_month + dt.timedelta(days=1) - relativedelta(years = 1))
print("one_year_rolling_back_date")
print(one_year_rolling_back_date)

first_date_of_year = DsnpHelperFunction.first_date_of_year(today_str, output_type="string")
print("first_date_of_year")
print(first_date_of_year)
first_date_of_last_month_str = DsnpHelperFunction.first_date_of_last_month(today_str, output_type="string")
print("first_date_of_last_month")
print(first_date_of_last_month_str)

pulled_data_date = last_month
comp_report_data_month = last_month
hra_report_data_month = last_month
reporting_end_date = last_date_of_last_month_str
print("reporting_end_date")
print(reporting_end_date)

In [ ]:
storage_path = r'//mbip/medicarepBI/Projects/COE/DSNP/dsnp_data_storage/monthly_flat_files'
output_path = f'{storage_path}/{reporting_end_date}'

isExist = os.path.exists(output_path)
print(output_path)
if not isExist:
    # Create a new directory because it does not exist
     os.makedirs(output_path)
     print("The new directory is created!")
else:
      print("The folder already exist")
print("Output files will sit in the folder above.")

In [ ]:
#locate data dump folder
if new_pull:
    pull_vbc = snpQueries.pull_vbc(reporting_year=2025)
    pull_vbc.to_parquet(f"{output_path}/pull_vbc.parquet")
else:
    try:
        pull_vbc = pd.read_parquet(f"{output_path}/pull_vbc.parquet")
    except:
        pull_vbc = snpQueries.pull_vbc(reporting_year=2025)
        pull_vbc.to_parquet(f"{output_path}/pull_vbc.parquet")


In [ ]:
#Prepare call log data, one year back
pbg_group_clean_df = pbgGroupTransform.get_pbg_group_clean(pull_vbc)

In [ ]:
pbg_group_clean_df.head(2)

In [ ]:
#save it for monthly aggregation
pbg_group_clean_df.to_parquet(f"{output_path}/pbg_group_clean_df.parquet")

In [ ]:
print("Process time -- %s seconds" % (time.time() - start_time))